# Ann Arbor House Prices
## Grant Jason, SIADS 521 Assignment 3

In [16]:
import requests
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact, interactive, Dropdown, IntRangeSlider
from IPython.display import display, clear_output

In [ ]:
# Download the data, filter, and clean it
url = 'https://files.zillowstatic.com/research/public_csvs/zhvi/Neighborhood_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv?t=1742155647'
response = requests.get(url)

# Define downloads as the directory to save the file
writeable_directory = os.path.expanduser('~/Downloads')
file_path = os.path.join(writeable_directory, 'gajason Zillow Data')

with open(file_path, 'wb') as file:
    file.write(response.content)

df = pd.read_csv(file_path)

# Filter the data to only include Ann Arbor
filtered_df = df[df['City'] == 'Ann Arbor']

# Melt the data so all dates are in one column
melted_df = pd.melt(filtered_df, id_vars=['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName', 'State', 'City' , 'Metro', 'CountyName'], 
                    var_name='Date', value_name='HomeValue')

cleaned_file_path = os.path.join(writeable_directory, 'cleaned_gajason_Zillow_Data.csv')
melted_df.to_csv(cleaned_file_path, index=False)